In [69]:
import os
import numpy as np
input_fname = 'onegin.txt'
output_fname = 'onegin_output.txt'
model_fname = 'onegin_model'

In [70]:
START_CHAR = '\b'
END_CHAR = '\t'
PADDING_CHAR = '\a'
chars = set([START_CHAR, '\n', END_CHAR])
with open(input_fname, encoding='utf8') as f:
    for line in f:
        chars.update(list(line.strip().lower()))
char_indices = { c : i for i, c in enumerate(sorted(list(chars))) }
char_indices[PADDING_CHAR] = 0
indices_to_chars = { i : c for c, i in char_indices.items() }
num_chars = len(chars)

In [71]:
def get_one(i, sz):
    res = np.zeros(sz)
    res[i] = 1
    return res
char_vectors = {
    c : (np.zeros(num_chars) if c == PADDING_CHAR else get_one(v, num_chars))
    for c,v in char_indices.items()
}
sentence_end_markers = set('.!?')
sentences = []
current_sentence = ''
with open(input_fname, encoding='utf8') as f:
    for line in f:
        s = line.strip().lower()
        if len(s) > 0:
            current_sentence += s + '\n'
        if len(s) == 0 or s[-1] in sentence_end_markers:
            current_sentence = current_sentence.strip()
            if len(current_sentence) > 10 and not current_sentence.startswith('. . . . . . . .'):
                sentences.append(current_sentence)
            current_sentence = ''

In [72]:
def get_matrices(sentences):
    max_sentence_len = np.max([len(x) for x in sentences])
    X = np.zeros((len(sentences), max_sentence_len, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), max_sentence_len, len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        char_seq = (START_CHAR + sentence + END_CHAR).ljust(max_sentence_len + 1, PADDING_CHAR)
        for t in range(max_sentence_len):
            X[i, t, :] = char_vectors[char_seq[t]]
            y[i, t, :] = char_vectors[char_seq[t + 1]]
        return X, y

In [73]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, TimeDistributed, Activation
from keras.optimizers import Adam
model = Sequential()
model.add(LSTM(output_dim=128, activation='tanh', return_sequences=True, input_dim=num_chars))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(output_dim=num_chars)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(clipnorm=1.), metrics=['accuracy'])

C:\Users\Avsenin.Pavel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
C:\Users\Avsenin.Pavel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="tanh", return_sequences=True, input_shape=(None, 84), units=128)`
  """
C:\Users\Avsenin.Pavel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=84)`
  import sys


In [74]:
test_indices = np.random.choice(range(len(sentences)), int(len(sentences) * 0.05))
sentences_train = [sentences[x] for x in set(range(len(sentences)))- set(test_indices)]
sentences_test = [sentences[x] for x in test_indices]
sentences_train = sorted(sentences_train, key = lambda x: len(x))
X_test, y_test = get_matrices(sentences_test)
batch_size = 16
def generate_batch():
    while True:
        for i in range(int(len(sentences_train) / batch_size)):
            sentences_batch = sentences_train[i*batch_size : (i+1)*batch_size]
            yield get_matrices(sentences_batch)

In [93]:
from keras.callbacks import ModelCheckpoint, CSVLogger, Callback
class CharSampler(Callback):
    def __init__(self, char_vectors, model):
        self.char_vectors = char_vectors
        self.model = model
    def on_train_begin(self, logs={}):
        self.epoch = 0
        if os.path.isfile(output_fname):
            os.remove(output_fname)
    def sample(self, preds, temperature=1.0):
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1)
        return np.argmax(probas)
    def sample_one(self, T):
        result = START_CHAR
        while len(result) < 500:
            Xsampled = np.zeros((1, len(result), num_chars))
            for t, c in enumerate(list(result)):
                Xsampled[0,t,:] = self.char_vectors[c]
            ysampled = self.model.predict(Xsampled, batch_size=1)[0,:]
            yv = ysampled[len(result)-1,:]
            selected_char = indices_to_chars[self.sample(yv, T)]
            if selected_char == END_CHAR:
                break
            result = result + selected_char
        return result
    def on_epoch_end(self, batch, logs={}):
        self.epoch = self.epoch+1
        if self.epoch % 1 == 0:
            print("\nEpoch %d text sampling:" % self.epoch)
            with open(output_fname, 'a', encoding='utf-8') as outf:
                outf.write('\n==== Epoch %d ====\n' % self.epoch)
                for T in [0.3, 0.5, 0.7, 0.9, 1.1]:
                    print('tsampling, T = %.1f...' % T)
                    for _ in range(5):
                        self.model.reset_states()
                        res = self.sample_one(T)
                        outf.write('\nT = %.1f\n%s\n' % (T, res[1:]))

In [113]:
cb_sampler = CharSampler(char_vectors, model)
cb_logger = CSVLogger('sin_l/' + model_fname + '.log')

In [95]:
model.fit_generator(generate_batch(),
    int(len(sentences_train) / batch_size) * batch_size,
    epochs=3, verbose=True, validation_data = (X_test, y_test),
    callbacks=[cb_sampler])

Epoch 1/3
1168/1168 [==============================] - 127s 109ms/step - loss: 0.0395 - acc: 0.0490 - val_loss: 0.0138 - val_acc: 0.0010

Epoch 1 text sampling:
tsampling, T = 0.3...
tsampling, T = 0.5...
tsampling, T = 0.7...
tsampling, T = 0.9...
tsampling, T = 1.1...
Epoch 2/3
1168/1168 [==============================] - 120s 103ms/step - loss: 0.0351 - acc: 0.0503 - val_loss: 0.0149 - val_acc: 0.0011

Epoch 2 text sampling:
tsampling, T = 0.3...
tsampling, T = 0.5...
tsampling, T = 0.7...
tsampling, T = 0.9...
tsampling, T = 1.1...
Epoch 3/3
1168/1168 [==============================] - 122s 105ms/step - loss: 0.0317 - acc: 0.0512 - val_loss: 0.0152 - val_acc: 9.1383e-04

Epoch 3 text sampling:
tsampling, T = 0.3...
tsampling, T = 0.5...
tsampling, T = 0.7...
tsampling, T = 0.9...
tsampling, T = 1.1...


In [116]:
from keras.layers import merge, Input
from keras.models import Model

vec = Input(shape=(None, num_chars))
l1 = LSTM(output_dim=128, activation='tanh', return_sequences=True)(vec)
l1_d = Dropout(0.2)(l1)

input2 = merge.Concatenate()([vec, l1_d])
l2 = LSTM(output_dim=128, activation='tanh', return_sequences=True)(input2)
l2_d = Dropout(0.2)(l2)

input3 = merge.Concatenate()([vec, l2_d])
l3 = LSTM(output_dim=128, activation='tanh', return_sequences=True)(input3)
l3_d = Dropout(0.2)(l3)

input_d = merge.Concatenate()([l1_d, l2_d, l3_d])
dense3 = TimeDistributed(Dense(output_dim=num_chars))(input_d)
output_res = Activation('softmax')(dense3)
model = Model(input=vec, output=output_res)
model.compile(loss='categorical_crossentropy', optimizer=Adam(clipnorm=1.), metrics=['accuracy'])

C:\Users\Avsenin.Pavel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="tanh", return_sequences=True, units=128)`
  """
C:\Users\Avsenin.Pavel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="tanh", return_sequences=True, units=128)`
  if __name__ == '__main__':
C:\Users\Avsenin.Pavel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="tanh", return_sequences=True, units=128)`
  del sys.path[0]
C:\Users\Avsenin.Pavel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=84)`
C:\Users\Avsenin.Pavel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Upda

In [ ]:
model.fit_generator(generate_batch(),
    int(len(sentences_train) / batch_size) * batch_size,
    epochs=3, verbose=True, validation_data = (X_test, y_test),
    callbacks=[cb_sampler])

Epoch 1/3
1168/1168 [==============================] - 306s 262ms/step - loss: 0.1637 - acc: 0.0155 - val_loss: 0.0098 - val_acc: 0.0012

Epoch 1 text sampling:
tsampling, T = 0.3...
tsampling, T = 0.5...
tsampling, T = 0.7...
tsampling, T = 0.9...
tsampling, T = 1.1...
Epoch 2/3
 426/1168 [=========>....................] - ETA: 3:20 - loss: 0.1255 - acc: 0.0234